<a href="https://colab.research.google.com/github/varunSinghGIT/Git_training/blob/main/Model%20train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np
import os
import json
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
!unzip Fruits_Classification.zip -d Fruits_Classification

Streaming output truncated to the last 5000 lines.
  inflating: Fruits_Classification/Fruits Classification/Grape/Grape (1765).jpeg  
  inflating: Fruits_Classification/Fruits Classification/Grape/Grape (1766).jpeg  
  inflating: Fruits_Classification/Fruits Classification/Grape/Grape (1767).jpeg  
  inflating: Fruits_Classification/Fruits Classification/Grape/Grape (1768).jpeg  
  inflating: Fruits_Classification/Fruits Classification/Grape/Grape (1769).jpeg  
  inflating: Fruits_Classification/Fruits Classification/Grape/Grape (177).jpeg  
  inflating: Fruits_Classification/Fruits Classification/Grape/Grape (1770).jpeg  
  inflating: Fruits_Classification/Fruits Classification/Grape/Grape (1771).jpeg  
  inflating: Fruits_Classification/Fruits Classification/Grape/Grape (1772).jpeg  
  inflating: Fruits_Classification/Fruits Classification/Grape/Grape (1774).jpeg  
  inflating: Fruits_Classification/Fruits Classification/Grape/Grape (1775).jpeg  
  inflating: Fruits_Classification/Fr

In [88]:
data = '/content/Fruits_Classification/Fruits Classification'
class_list = os.listdir(data)
class_count= len(data)

In [89]:
class_list
#['Apple', 'Banana', 'Lemon', 'Stawberry']

['2', '.ipynb_checkpoints', '1', '4', '3', '0']

In [90]:
images = []  # list of images
class_no = [] # list of classes

In [91]:
for x in range (0,5):
    myPiclist= os.listdir(data+"/"+str(x))
    for y in myPiclist:
        img = cv2.imread(data+"/"+str(x)+"/"+y)
        img = cv2.resize(img,(32,32))
        images.append(img)
        class_no.append(x)

    print(x,"Completed....")

0 Completed....
1 Completed....
2 Completed....
3 Completed....
4 Completed....


In [92]:
images = np.array(images)
class_no = np.array(class_no)

# Train Test Split

In [93]:
from sklearn.model_selection import train_test_split

In [94]:
help(train_test_split) # more info about spliting

Help on function train_test_split in module sklearn.model_selection._split:

train_test_split(*arrays, test_size=None, train_size=None, random_state=None, shuffle=True, stratify=None)
    Split arrays or matrices into random train and test subsets.
    
    Quick utility that wraps input validation,
    ``next(ShuffleSplit().split(X, y))``, and application to input data
    into a single call for splitting (and optionally subsampling) data into a
    one-liner.
    
    Read more in the :ref:`User Guide <cross_validation>`.
    
    Parameters
    ----------
    *arrays : sequence of indexables with same length / shape[0]
        Allowed inputs are lists, numpy arrays, scipy-sparse
        matrices or pandas dataframes.
    
    test_size : float or int, default=None
        If float, should be between 0.0 and 1.0 and represent the proportion
        of the dataset to include in the test split. If int, represents the
        absolute number of test samples. If None, the value is set to

In [95]:
X_train,X_test,y_train,y_test = train_test_split(images,class_no,test_size=0.2)
X_train,X_validation,y_train,y_validation = train_test_split(X_train,y_train,test_size=0.2)

In [96]:
print(X_train.shape) # 3309 train images
print(X_test.shape)  # 1035 test images             # rgb color channel
print(X_validation.shape) # 828 # validation images

(6208, 32, 32, 3)
(1940, 32, 32, 3)
(1552, 32, 32, 3)


In [97]:
# Preprossessing function
def prepos(img):
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img = cv2.equalizeHist(img)
    img = img/255

    return img

In [98]:
X_train= np.array(list(map(prepos,X_train)))
X_test= np.array(list(map(prepos,X_test)))
X_validation= np.array(list(map(prepos,X_validation)))

In [99]:
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],X_train.shape[2],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],X_test.shape[2],1)
X_validation = X_validation.reshape(X_validation.shape[0],X_validation.shape[1],X_validation.shape[2],1)

# Preparing the Data for the model

In [100]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

In [101]:
help(ImageDataGenerator)

Help on class ImageDataGenerator in module keras.src.legacy.preprocessing.image:

class ImageDataGenerator(builtins.object)
 |  ImageDataGenerator(featurewise_center=False, samplewise_center=False, featurewise_std_normalization=False, samplewise_std_normalization=False, zca_whitening=False, zca_epsilon=1e-06, rotation_range=0, width_shift_range=0.0, height_shift_range=0.0, brightness_range=None, shear_range=0.0, zoom_range=0.0, channel_shift_range=0.0, fill_mode='nearest', cval=0.0, horizontal_flip=False, vertical_flip=False, rescale=None, preprocessing_function=None, data_format=None, validation_split=0.0, interpolation_order=1, dtype=None)
 |  
 |  DEPRECATED.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, featurewise_center=False, samplewise_center=False, featurewise_std_normalization=False, samplewise_std_normalization=False, zca_whitening=False, zca_epsilon=1e-06, rotation_range=0, width_shift_range=0.0, height_shift_range=0.0, brightness_range=None, shear_range=0.0, zoom_

In [102]:
dataGen = ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    shear_range=0.1,
    rotation_range=10,
    brightness_range=[0.2, 1.5],  # Added brightness adjustment
    horizontal_flip=True,
    fill_mode='nearest'
    )
dataGen.fit(X_train)

In [103]:
y_train = to_categorical(y_train,class_count)
y_test = to_categorical(y_test,class_count)
y_validation = to_categorical(y_validation,class_count)

# Creating the Model

In [104]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from keras.optimizers import Adam


In [105]:
imageDimensions= (32,32,3)

In [106]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense,BatchNormalization,Dropout

In [127]:
def myModel():
    noOfFilters = 128
    sizeOfFilter1 = (5, 5)
    sizeOfFilter2 = (3, 3)
    sizeOfPool = (2, 2)
    noOfNodes = 512

    model = Sequential()
    model.add(Conv2D(noOfFilters, sizeOfFilter1, input_shape=(imageDimensions[0], imageDimensions[1], 1), activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(noOfFilters, sizeOfFilter1, activation='relu'))
    model.add(MaxPooling2D(pool_size=sizeOfPool))
    model.add(Conv2D(noOfFilters, sizeOfFilter2, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(noOfFilters // 2, sizeOfFilter2, activation='relu'))
    model.add(Conv2D(noOfFilters // 2, sizeOfFilter2, activation='relu'))
    model.add(MaxPooling2D(pool_size=sizeOfPool))
    model.add(Dropout(0.3))  # Increased dropout for better regularization

    model.add(Flatten())
    model.add(Dense(noOfNodes, activation='relu'))
    model.add(Dropout(0.3))  # Increased dropout for better regularization
    model.add(Dense(class_count, activation='softmax'))

    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [128]:
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Dropout, Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [129]:
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.00001)

In [130]:
model= myModel()
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_51 (Conv2D)                   │ (None, 28, 28, 128)         │           3,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_38               │ (None, 28, 28, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_52 (Conv2D)                   │ (None, 24, 24, 128)         │         409,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_32 (MaxPooling2D)      │ (None, 12, 12, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_53 (Conv2D)                   │ (None, 10, 10, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_39               │ (None, 10, 10, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_54 (Conv2D)                   │ (None, 8, 8, 64)            │          73,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_55 (Conv2D)                   │ (None, 6, 6, 64)            │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_33 (MaxPooling2D)      │ (None, 3, 3, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_17 (Dropout)                 │ (None, 3, 3, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_9 (Flatten)                  │ (None, 576)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_24 (Dense)                     │ (None, 512)                 │         295,424 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_18 (Dropout)                 │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_25 (Dense)                     │ (None, 52)                  │          26,676 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 994,484 (3.79 MB)

 Trainable params: 993,972 (3.79 MB)

 Non-trainable params: 512 (2.00 KB)

In [132]:
history = model.fit(
    dataGen.flow(X_train, y_train, batch_size=32),
    steps_per_epoch=100,
    epochs=20,
    validation_data=(X_validation, y_validation),
    shuffle=True
)


Epoch 1/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 4s 37ms/step - accuracy: 0.2094 - loss: 1.6137 - val_accuracy: 0.2171 - val_loss: 2.2938
Epoch 2/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 4s 36ms/step - accuracy: 0.2006 - loss: 1.6208 - val_accuracy: 0.2055 - val_loss: 2.2965
Epoch 3/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 6s 33ms/step - accuracy: 0.1904 - loss: 1.6290 - val_accuracy: 0.1972 - val_loss: 2.1439
Epoch 4/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 3s 29ms/step - accuracy: 0.2219 - loss: 1.6166 - val_accuracy: 0.1914 - val_loss: 1.9876
Epoch 5/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 7s 31ms/step - accuracy: 0.1878 - loss: 1.6227 - val_accuracy: 0.2068 - val_loss: 1.6707
Epoch 6/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - accuracy: 0.1993 - loss: 1.6129 - val_accuracy: 0.2120 - val_loss: 1.6732
Epoch 7/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 8s 33ms/step - accuracy: 0.1927 - loss: 1.6218 - val_accuracy: 0.1965 - val_loss: 1.6959
Epoch 8/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - accuracy: 0.2024 - loss: 1.6220 - val_accu

In [38]:
score = model.evaluate(X_test,y_test,verbose=0)
print('Test Score = ',score[0])
print('Test Accuracy =', score[1])

Test Score =  1.7586681842803955
Test Accuracy = 0.4103092849254608


In [ ]:
# save the model
model_json = model.to_json()
with open("model-bw.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights('model-bw.h5')